In [4]:
# Import librairies

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go


In [5]:
variables_etude = pd.read_csv('https://medical-deserts.s3.eu-west-3.amazonaws.com/variables_etude.csv')

#Preprocessing dataset

In [6]:
variables_etude_work = variables_etude.copy()

In [7]:
variables_etude_work.head()

,Code commune,region_geojson_name,Population en 2014 (princ),Catégorie commune dans aire d'attraction des villes 2020,Tranche détaillée d'aire d'attraction des villes 2020,Libellé degré de densité,APL aux médecins généralistes (sans borne d'âge),densité 4 niveaux
0,01001,Auvergne-Rhône-Alpes,767.0,Commune de la couronne,Aire de moins de 10 000 habitants,Rural à habitat dispersé,"2,293",Commune peu dense
1,01002,Auvergne-Rhône-Alpes,239.0,Commune hors attraction des pôles,Commune hors attraction des villes,Rural à habitat dispersé,"2,6",Commune très peu dense
2,01004,Auvergne-Rhône-Alpes,14022.0,Commune-centre,Aire de 30 000 à moins de 50 000 habitants,Centres urbains intermédiaires,"4,079",Commune de densité intermédiaire
3,01005,Auvergne-Rhône-Alpes,1627.0,Commune de la couronne,Aire de 1 000 000 d’habitants ou plus (hors Pa...,Bourgs ruraux,"4,378",Commune peu dense
4,01006,Auvergne-Rhône-Alpes,109.0,Commune de la couronne,Aire de 20 000 à moins de 30 000 habitants,Rural à habitat dispersé,"1,069",Commune très peu dense


In [8]:
variables_etude_work = variables_etude_work[~variables_etude_work['Code commune'].str.startswith(('97', '98'))]

In [9]:
variables_etude_work.drop_duplicates(inplace = True)

In [10]:
variables_etude_work["APL aux médecins généralistes (sans borne d'âge)"] = variables_etude_work["APL aux médecins généralistes (sans borne d'âge)"].str.replace(',', '.').astype(float)

In [11]:
print('Number of rows :', variables_etude_work.shape[0])
print('Number of columns :', variables_etude_work.shape[1])
print()

# Dataset statistics
print('Basics statistics :')
summary_stats_all = variables_etude_work.describe(include='all')
display(summary_stats_all)
print()

# Missing values percentage
missing_percentages = (variables_etude_work.isna().mean() * 100).round(2)
print('Percentage of missing values: ')
print(missing_percentages)

Number of rows : 34869
Number of columns : 8

Basics statistics :


,Code commune,region_geojson_name,Population en 2014 (princ),Catégorie commune dans aire d'attraction des villes 2020,Tranche détaillée d'aire d'attraction des villes 2020,Libellé degré de densité,APL aux médecins généralistes (sans borne d'âge),densité 4 niveaux
count,34869,34869,34819.000000,34823,34823,34823,34862.000000,34823
unique,34868,13,NaN,5,17,7,NaN,4
top,39274,Grand Est,NaN,Commune de la couronne,Commune hors attraction des villes,Rural à habitat dispersé,NaN,Commune peu dense
freq,2,5121,NaN,24203,8907,18358,NaN,18963
mean,NaN,NaN,1819.161033,NaN,NaN,NaN,3.275459,NaN
std,NaN,NaN,8356.397126,NaN,NaN,NaN,1.281371,NaN
min,NaN,NaN,1.000000,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,199.000000,NaN,NaN,NaN,2.491000,NaN
50%,NaN,NaN,449.000000,NaN,NaN,NaN,3.216000,NaN
75%,NaN,NaN,1124.500000,NaN,NaN,NaN,3.996000,NaN



Percentage of missing values: 
Code commune                                                0.00
region_geojson_name                                         0.00
Population en 2014 (princ)                                  0.14
Catégorie commune dans aire d'attraction des villes 2020    0.13
Tranche détaillée d'aire d'attraction des villes 2020       0.13
Libellé degré de densité                                    0.13
APL aux médecins généralistes (sans borne d'âge)            0.02
densité 4 niveaux                                           0.13
dtype: float64


In [12]:
variables_etude_work.dropna(inplace= True)

In [13]:
variables_etude_work.drop('Code commune', inplace = True, axis = 1)

In [14]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "APL aux médecins généralistes (sans borne d'âge)"

X = variables_etude_work.drop(target_variable, axis = 1)
Y = variables_etude_work.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    2.293
1    2.600
2    4.079
3    4.378
4    1.069
Name: APL aux médecins généralistes (sans borne d'âge), dtype: float64

X :
    region_geojson_name  Population en 2014 (princ)  \
0  Auvergne-Rhône-Alpes                       767.0   
1  Auvergne-Rhône-Alpes                       239.0   
2  Auvergne-Rhône-Alpes                     14022.0   
3  Auvergne-Rhône-Alpes                      1627.0   
4  Auvergne-Rhône-Alpes                       109.0   

  Catégorie commune dans aire d'attraction des villes 2020  \
0                             Commune de la couronne         
1                  Commune hors attraction des pôles         
2                                     Commune-centre         
3                             Commune de la couronne         
4                             Commune de la couronne         

  Tranche détaillée d'aire d'attraction des villes 2020  \
0                  Aire de moins de 10 000 habitants    

In [15]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [16]:
# Automatically detect names of numeric/categorical columns
numeric_features = []
categorical_features = []
for i, t in X.dtypes.items():
    if ('float' in str(t)) or ('int' in str(t)):
        numeric_features.append(i)
    else:
        categorical_features.append(i)
print('Found numeric features ', numeric_features)
print('Found categorical features ', categorical_features)

Found numeric features  ['Population en 2014 (princ)']
Found categorical features  ['region_geojson_name', "Catégorie commune dans aire d'attraction des villes 2020", "Tranche détaillée d'aire d'attraction des villes 2020", 'Libellé degré de densité', 'densité 4 niveaux']


In [17]:
#Multiple Transformations preprocessing

# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Create pipeline for categorical features
categorical_transformer = Pipeline(
    steps=[
    ('encoder', OneHotEncoder(drop='first', handle_unknown='ignore')) # first column will be dropped to avoid creating correlations between features
    ])

# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

In [18]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) # Don't fit again !!
print('...Done.')
print(X_test[0:5,:])
print()

Performing preprocessings on train set...
        region_geojson_name  Population en 2014 (princ)  \
2418   Auvergne-Rhône-Alpes                       270.0   
17535  Auvergne-Rhône-Alpes                      5669.0   
13603             Occitanie                       178.0   
15876  Auvergne-Rhône-Alpes                       957.0   
27871    Nouvelle-Aquitaine                      6575.0   

      Catégorie commune dans aire d'attraction des villes 2020  \
2418                   Commune hors attraction des pôles         
17535                             Commune de la couronne         
13603                             Commune de la couronne         
15876                  Commune hors attraction des pôles         
27871                             Commune de la couronne         

      Tranche détaillée d'aire d'attraction des villes 2020  \
2418                  Commune hors attraction des villes      
17535       Aire de 400 000 à moins de 500 000 habitants      
13603  Aire de 1 

...Done.
  (0, 0)	-0.19243184556414164
  (0, 15)	1.0
  (0, 32)	1.0
  (0, 38)	1.0
  (0, 41)	1.0
  (1, 0)	0.5249035728745958
  (1, 14)	1.0
  (1, 25)	1.0
  (1, 33)	1.0
  (2, 0)	-0.20465537927341446
  (2, 9)	1.0
  (2, 14)	1.0
  (2, 38)	1.0
  (2, 41)	1.0
  (3, 0)	-0.10115393623511543
  (3, 15)	1.0
  (3, 32)	1.0
  (3, 37)	1.0
  (3, 40)	1.0
  (4, 0)	0.6452788070115649
  (4, 8)	1.0
  (4, 14)	1.0
  (4, 22)	1.0
  (4, 33)	1.0

Performing preprocessings on test set...
           region_geojson_name  Population en 2014 (princ)  \
30397  Bourgogne-Franche-Comté                      1589.0   
28893                Occitanie                       261.0   
5846      Auvergne-Rhône-Alpes                       278.0   
16443  Bourgogne-Franche-Comté                       183.0   
7762   Bourgogne-Franche-Comté                       109.0   

      Catégorie commune dans aire d'attraction des villes 2020  \
30397                             Commune de la couronne         
28893                             

Linear regression 

In [19]:
# Train model
model = LinearRegression() # or LniearRegression pour regression lineaire par exemple
print("Training model...")
model.fit(X_train, Y_train)  # Training is always done on train set !!
print("...Done.")

Training model...
...Done.


In [20]:
# Print R^2 scores
print("R2 score on training set : ", model.score(X_train, Y_train))
print("R2 score on test set : ", model.score(X_test, Y_test))

R2 score on training set :  0.19058583704241172
R2 score on test set :  0.1875988784026037


Random Forest 

In [21]:
# Instanciate RandomForestRegressor
rf = RandomForestRegressor()

In [22]:
rf.fit(X_train, Y_train)

RandomForestRegressor()

In [24]:
# Print R^2 scores
print("R2 score on training set : ", rf.score(X_train, Y_train))
print("R2 score on test set : ", rf.score(X_test, Y_test))

R2 score on training set :  0.7989110855923712
R2 score on test set :  0.019121214500856132
